Channel PCA captures most significant patterns across frames for each channel, potentially highlighting the most prominent changes or features in the video content for that specific channel.
**Do we want that or are there better ways to capture spatial features per frame?**

In [1]:
import os
import pickle
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, IncrementalPCA
from tqdm import tqdm

In [2]:
def find_repo_root(path='.'):
    path = os.path.abspath(path)
    while not os.path.isdir(os.path.join(path, '.git')):
        parent = os.path.dirname(path)
        if parent == path:
            # We've reached the root of the file system without finding '.git'
            return None
        path = parent
    return path

repo_root = find_repo_root()
print("Repository Root:", repo_root)

Repository Root: None


In [3]:
def get_full_path(relative_path, repo_root):
    if not repo_root:
        raise ValueError("Repository root not found. Ensure you're inside a Git repository.")

    return os.path.join(repo_root, relative_path)


In [4]:
# step 1 function
def load_and_combine_tensors(stage_name, input_folder, num_videos):
    combined_tensor = []
    video_indices = {}

    for video_id in range(1, num_videos + 1):
        filename = f"{str(video_id).zfill(4)}_{stage_name}.pkl"
        file_path = os.path.join(input_folder, stage_name, filename)

        if os.path.exists(file_path):
            #print(f"Loading tensor from: {file_path}")
            with open(file_path, 'rb') as file:
                tensor = pickle.load(file)
                combined_tensor.append(tensor)
                # Track start and end indices for each video
                end_index = sum(t.shape[0] for t in combined_tensor)
                video_indices[str(video_id).zfill(4)] = (end_index - tensor.shape[0], end_index)

    if not combined_tensor:
        print("No tensors found to combine.")
        return None, None

    combined_tensor = np.concatenate(combined_tensor, axis=0)
    return combined_tensor, video_indices


In [5]:
# Step 2: globalized standardization (only based on training set)
def standardize_tensors(combined_tensor, video_indices, training_end_id='0800'):
    reshaped_tensor = combined_tensor.reshape(combined_tensor.shape[0], -1)
    scaler = StandardScaler()

    # Find the end index of the training set
    training_end_index = video_indices[training_end_id][1]

    # Fit the scaler only on the training set
    scaler.fit(reshaped_tensor[:training_end_index])

    # Transform both training and test sets
    standardized_data = scaler.transform(reshaped_tensor)
    
    return standardized_data.reshape(combined_tensor.shape)

In [6]:
# Function to slice feature maps into four equal parts
def slice_feature_maps(feature_maps):
    _, _, height, width, _ = feature_maps.shape

    # Calculate midpoints
    mid_height = height // 2
    mid_width = width // 2

    # Slicing the feature maps into four equal parts
    top_left = feature_maps[:, :, :mid_height, :mid_width, :]
    top_right = feature_maps[:, :, :mid_height, mid_width:, :]
    bottom_left = feature_maps[:, :, mid_height:, :mid_width, :]
    bottom_right = feature_maps[:, :, mid_height:, mid_width:, :]

    return [top_left, top_right, bottom_left, bottom_right]

In [15]:
def apply_rpca_and_save(sliced_tensors, stage_name, output_folder, n_components, training_end_id=800, seed=42):
    
    pca_results = []
    for idx , feature_maps in enumerate(tqdm(sliced_tensors, desc="Performing PCA...")):
        # Reshape the slice to 2D array, concatenate the feature maps for each 30-frame segment
        n_videos = feature_maps.shape[0] // 30
        reshaped_data = np.zeros((n_videos, feature_maps.shape[2] * feature_maps.shape[3] * feature_maps.shape[4] * 30))

        for i in range(n_videos):
            # Flatten and concatenate the feature maps for each segment
            video = feature_maps[i*30:(i+1)*30].reshape(-1)
            reshaped_data[i, :] = video
        
        print(f"Shape of flattened slice {idx}: {reshaped_data.shape}")
        
        # Apply PCA
        # stage_2 and stage_3 need incremental PCA bc of RAM restrictions
        if stage_name in ["stage_2", "stage_3"]:
            # Apply Incremental PCA with increasing number of components until 95% variance is reached
            print("Fitting IPCA...")
            
            # n_components_ratio = n_components
            # cumulative_variance = 1
            # n_components = 800

            # batch_size = 100  # Example batch size, adjust based on your memory constraints
            # n_batches = training_end_id // batch_size
            
            # while cumulative_variance >= n_components_ratio:
            #     n_components -= 1
            #     ipca = IncrementalPCA(n_components=n_components)
                
            #     for batch_idx in range(n_batches):
            #         start_idx = batch_idx * batch_size
            #         end_idx = start_idx + batch_size
            #         ipca.partial_fit(reshaped_data[start_idx:end_idx, :])

            #     cumulative_variance = sum(ipca.explained_variance_ratio_)
            #     print(f"Variance with {n_components} components: {cumulative_variance}")


            ipca = IncrementalPCA(n_components=720)
            ipca.fit(reshaped_data[:training_end_id, :])
            cumulative_variance = sum(ipca.explained_variance_ratio_)
            print(f"Variance with {n_components} components: {cumulative_variance}")
            pca = ipca
            print("Transforming IPCA...")
        
        else:
            pca = PCA(n_components=n_components, random_state=seed)
            print("Fitting PCA...")
            pca.fit(reshaped_data[:training_end_id, :])
            print("Transforming PCA...")

        pca_result = pca.transform(reshaped_data)
        print(f"Number of PCs in slice {idx}: {pca_result.shape}")
        print(f"Variance captured by PCs: {sum(pca.explained_variance_ratio_)}")
        # Append the PCA result
        pca_results.append(pca_result)

    # Concatenate the PCA results from all slices
    final_pca_results = np.concatenate(pca_results, axis=1)
    print(final_pca_results.shape)
    # save PCA results
    pca_folder = os.path.join(output_folder, f"PCA_slice_{n_components}", stage_name)
    if not os.path.exists(pca_folder):
        os.makedirs(pca_folder)
    
    stage_number = stage_name[-1]
    file_path = os.path.join(pca_folder, f'layer_{stage_number}_pca.pkl')
    with open(file_path, 'wb') as f:
        pickle.dump(final_pca_results, f)
    
    print(f"{stage_name} PCs stored in: {file_path}")

In [16]:
def process_stage_for_pca(input_folder, output_folder, stage_name):
    """
    Process all videos of a given stage: standardize, apply PCA, and save the PCA-transformed tensors.
    Args:
    - input_folder: Folder containing the pre-processed videos.
    - output_folder: Folder to save PCA results.
    - stage_name: Name of the stage to process.
    Returns:
    - DataFrame containing metadata (video ID and variance captured).
    """
     # Use the current working directory or a known absolute path
    current_working_directory = os.getcwd()
    stage_folder = os.path.join(current_working_directory, input_folder, stage_name)
    print("Attempting to access:", stage_folder)

    if not os.path.exists(stage_folder):
        print("Directory not found:", stage_folder)
        return None
    # Calculate the number of video files in the folder
    num_videos = len([f for f in os.listdir(stage_folder) if os.path.isfile(os.path.join(stage_folder, f))])
    print(f"Number of videos found: {num_videos}")

    # Step 1: Load and combine tensors
    if stage_name not in ["stage_2", "stage_3"]:
        combined_tensor, video_indices = load_and_combine_tensors(stage_name, input_folder, num_videos)
    print("Step 1 done.")
    
    # Step 2: Globally standardize the tensor
    if stage_name in ["stage_2", "stage_3"]:
        st_folder = os.path.join(output_folder, f"fm_standardized", stage_name)
        file_path = os.path.join(st_folder, f'fm_standardized_{stage_name}.pkl')
        with open(file_path, 'rb') as f:
            standardized_tensor = pickle.load(f)
    else:
        standardized_tensor = standardize_tensors(combined_tensor, video_indices)
        # # save standardized tensors
        # st_folder = os.path.join(output_folder, f"fm_standardized", stage_name)
        # if not os.path.exists(st_folder):
        #     os.makedirs(st_folder)
        # file_path = os.path.join(st_folder, f'fm_standardized_{stage_name}.pkl')
        # with open(file_path, 'wb') as f:
        #     pickle.dump(standardized_tensor, f)
        # print(f"{stage_name} standardized tensors stored in: {file_path}")
        del combined_tensor, video_indices
    print("Step 2 done.")

    # Step 3: Apply PCA to each tensor and save the result
    sliced_tensors = slice_feature_maps(standardized_tensor)
    del standardized_tensor
    apply_rpca_and_save(sliced_tensors, stage_name, output_folder, n_components)
    print("Step 3 done.")

In [13]:
print(os.getcwd())
print(repo_root)

/work
None


In [14]:
# Example usage
input_folder = 'preprocessed_videos_30frames'
output_folder = os.getcwd()
stages = ["stage_3"] # ["stage_2", "stage_3", "stage_4", "stage_5"] # success: ["stage_1"]
n_components = 0.95

# Iterate over each stage and process it
for stage in stages:
    print(f"Processing {stage}...")
    process_stage_for_pca(input_folder, output_folder, stage)

Processing stage_2...
Attempting to access: /work/preprocessed_videos_30frames/stage_2
Number of videos found: 1000
Step 1 done.
stage_2 standardized tensors stored in: /work/fm_standardized/stage_2/fm_standardized_stage_2.pkl
Step 2 done.


In [ ]:
#-----------------------------------------------------------------------

In [8]:
# zip pca folder
import shutil
directory_to_zip = "PCA_slice_0.95"  # Replace with your directory name
output_filename = "PCA_slice_0.95_dataset"  # Replace with your desired output name
output_path = os.path.join(os.getcwd(), output_filename)
shutil.make_archive(output_path, 'zip', directory_to_zip)

'/work/PCA_slice_0.95_dataset.zip'

In [ ]:
stage = "stage_1"
stage_nr = stage[-1]
path = f"PCA_slice_0.95/{stage}/layer_{stage_nr}_pca.pkl"
with open(path, 'rb') as f:
    pcs = pickle.load(f)

print(pcs.shape)

In [9]:
stage_name = "stage_3"
output_folder = os.getcwd()
if stage_name in ["stage_2", "stage_3"]:
    st_folder = os.path.join(output_folder, f"fm_standardized", stage_name)
    file_path = os.path.join(st_folder, f'fm_standardized_{stage_name}.pkl')
    with open(file_path, 'rb') as f:
        standardized_tensor = pickle.load(f)

In [10]:
print(standardized_tensor.shape)

(30000, 1, 28, 28, 512)


In [11]:
sliced_tensors = slice_feature_maps(standardized_tensor)
del standardized_tensor
for slice in sliced_tensors:
    print(slice.shape)

(30000, 1, 14, 14, 512)
(30000, 1, 14, 14, 512)
(30000, 1, 14, 14, 512)
(30000, 1, 14, 14, 512)


In [12]:
for idx , feature_maps in enumerate(tqdm(sliced_tensors, desc="Performing PCA...")):
    # Reshape the slice to 2D array, concatenate the feature maps for each 30-frame segment
    n_videos = feature_maps.shape[0] // 30
    reshaped_data = np.zeros((n_videos, feature_maps.shape[2] * feature_maps.shape[3] * feature_maps.shape[4] * 30))

    for i in range(n_videos):
        # Flatten and concatenate the feature maps for each segment
        video = feature_maps[i*30:(i+1)*30].reshape(-1)
        reshaped_data[i, :] = video
    
    print(f"Shape of flattened slice {idx}: {reshaped_data.shape}")

Performing PCA...:  25%|██▌       | 1/4 [00:19<00:59, 19.69s/it]

Shape of flattened slice 0: (1000, 3010560)


Performing PCA...:  50%|█████     | 2/4 [00:40<00:40, 20.38s/it]

Shape of flattened slice 1: (1000, 3010560)


Performing PCA...:  75%|███████▌  | 3/4 [01:01<00:20, 20.56s/it]

Shape of flattened slice 2: (1000, 3010560)


Performing PCA...: 100%|██████████| 4/4 [01:21<00:00, 20.49s/it]

Shape of flattened slice 3: (1000, 3010560)


In [13]:
reshaped_data.shape

(1000, 3010560)

In [15]:
pca = PCA(n_components=720, svd_solver='randomized', random_state=42)
pca.fit(reshaped_data)

PCA(n_components=720, random_state=42, svd_solver='randomized')

In [16]:
np.sum(pca.explained_variance_ratio_)

0.846802380649253